In [1]:
import numpy as np
import datasets
import json
# import torch


In [3]:
dataset = datasets.load_dataset('csv', data_files='./combined_train.csv')
def format_dataset(example):
    """
    string to dictionary 
    dictionary became string when saved to csv. need to convert it back to dict.  
    """
    example['answers'] = json.loads(example['answers']) 
    if type(example['answers']) == str:
        example['answers'] = json.loads(example['answers']) 
    return example
dataset = dataset.map(format_dataset)

Found cached dataset csv (C:/Users/fgmal/.cache/huggingface/datasets/csv/default-3bf97719ae4c7c64/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\fgmal\.cache\huggingface\datasets\csv\default-3bf97719ae4c7c64\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-ab92c90d13642b4d.arrow


In [4]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [5]:
from transformers import DefaultDataCollator
from transformers import AutoTokenizer

data_collator = DefaultDataCollator()
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
import random
random.seed(42)
dataset = dataset["train"].select(
    random.sample(range(len(dataset["train"])), 50000)
).train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 5000
    })
})

In [7]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_dataset

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 5000
    })
})

In [9]:
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

C:\Users\fgmal\AppData\Roaming\Python\Python310\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/8439 [00:00<?, ?it/s]

{'loss': 2.3814, 'learning_rate': 1.8815025476952245e-05, 'epoch': 0.18}
{'loss': 1.5572, 'learning_rate': 1.7630050953904493e-05, 'epoch': 0.36}
{'loss': 1.3262, 'learning_rate': 1.644507643085674e-05, 'epoch': 0.53}
{'loss': 1.1784, 'learning_rate': 1.5260101907808983e-05, 'epoch': 0.71}
{'loss': 1.0984, 'learning_rate': 1.4075127384761229e-05, 'epoch': 0.89}


  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 0.8619016408920288, 'eval_runtime': 742.6859, 'eval_samples_per_second': 6.732, 'eval_steps_per_second': 0.421, 'epoch': 1.0}
{'loss': 0.9485, 'learning_rate': 1.2890152861713474e-05, 'epoch': 1.07}
{'loss': 0.8179, 'learning_rate': 1.170517833866572e-05, 'epoch': 1.24}
{'loss': 0.8223, 'learning_rate': 1.0520203815617964e-05, 'epoch': 1.42}
{'loss': 0.7982, 'learning_rate': 9.33522929257021e-06, 'epoch': 1.6}
{'loss': 0.7573, 'learning_rate': 8.150254769522456e-06, 'epoch': 1.78}
{'loss': 0.7749, 'learning_rate': 6.9652802464747016e-06, 'epoch': 1.96}


  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 0.7722708582878113, 'eval_runtime': 741.6471, 'eval_samples_per_second': 6.742, 'eval_steps_per_second': 0.422, 'epoch': 2.0}
{'loss': 0.65, 'learning_rate': 5.780305723426947e-06, 'epoch': 2.13}
{'loss': 0.5878, 'learning_rate': 4.5953312003791925e-06, 'epoch': 2.31}
{'loss': 0.5772, 'learning_rate': 3.410356677331438e-06, 'epoch': 2.49}
{'loss': 0.5906, 'learning_rate': 2.225382154283683e-06, 'epoch': 2.67}
{'loss': 0.5734, 'learning_rate': 1.0404076312359284e-06, 'epoch': 2.84}


  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 0.7858432531356812, 'eval_runtime': 740.299, 'eval_samples_per_second': 6.754, 'eval_steps_per_second': 0.423, 'epoch': 3.0}
{'train_runtime': 52566.0198, 'train_samples_per_second': 2.568, 'train_steps_per_second': 0.161, 'train_loss': 0.9444143650359149, 'epoch': 3.0}


TrainOutput(global_step=8439, training_loss=0.9444143650359149, metrics={'train_runtime': 52566.0198, 'train_samples_per_second': 2.568, 'train_steps_per_second': 0.161, 'train_loss': 0.9444143650359149, 'epoch': 3.0})

In [10]:
trainer.save_model("biased_squad")

## inference

In [12]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [13]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="./biased_squad")
question_answerer(question=question, context=context)

{'score': 0.8730721473693848, 'start': 93, 'end': 95, 'answer': '13'}

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./biased_squad")
inputs = tokenizer(question, context, return_tensors="pt")

In [17]:
from transformers import AutoModelForQuestionAnswering
import torch

model = AutoModelForQuestionAnswering.from_pretrained("./biased_squad")
with torch.no_grad():
    outputs = model(**inputs)

In [19]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [20]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'13'